# Pre processing and Tree Segmentation

In [6]:
from whitebox import WhiteboxTools
import laspy
import pdal
import json
import numpy as np
import sys
from datetime import datetime
import rasterio

from pathlib import Path

# set root directory
BASE_DIR = Path.cwd()

# Falls dein preprocessing-Ordner im Projekt liegt:
WORK_DIR = BASE_DIR / "preprocessing"

# Whitebox initialisieren
wbt = WhiteboxTools()
wbt.set_working_dir(str(WORK_DIR))

print("Working directory:", wbt.work_dir)

Decompressing WhiteboxTools_win_amd64.zip ...
WhiteboxTools package directory: c:\Users\c7161080\AppData\Local\miniforge3\envs\TreeSegmentation\Lib\site-packages\whitebox
Working directory: d:\Lukas_B\Masterarbeit\preprocessing


# Ground classification with Cloth Simulation Filter (CSF) [Zhang et al., 2016]

In [11]:
# --- Define folders ---
INPUT_FILE = WORK_DIR / "Wenns_Data" / "PC" / "Wenns_aoi.las"
OUTPUT_DIR = WORK_DIR / "output_lidar"

# --- Output ---
ALL_FILE = OUTPUT_DIR / "las_ground_nonground.las"
GROUND_FILE = OUTPUT_DIR / "ground_csf.las"
NONGROUND_FILE = OUTPUT_DIR / "nonground_csf.las"

# initialize json pipeline
json_pipeline = {
    "pipeline": [
        {
            "type": "readers.las",
            "filename": str(INPUT_FILE)
        },

        {
            "type": "filters.assign",
            "value": [
            "ReturnNumber = 1 WHERE ReturnNumber < 1",
            "NumberOfReturns = 1 WHERE NumberOfReturns < 1"
            ]
        },

        {
            "type": "filters.outlier"
        },
        {
            "type": "filters.csf",
            "resolution": 0.2,
        },
        # Export all
        {
            "type": "writers.las",
            "filename": str(ALL_FILE),
        },

        # Ground export
        {
            "type": "writers.las",
            "filename": str(GROUND_FILE),
            "where": "Classification == 2"
        },

        # Non-ground export
        {
            "type": "writers.las",
            "filename": str(NONGROUND_FILE),
            "where": "Classification != 2"
        }
    ]
}

json_pipeline = json.dumps(json_pipeline)

pipeline = pdal.Pipeline(json_pipeline)
count = pipeline.execute()

# Create DTM, DSM and CHM

In [14]:
las_ground_nonground = laspy.read("preprocessing/output_lidar/las_ground_nonground.las")

xmin, xmax = np.min(las_ground_nonground.x), np.max(las_ground_nonground.x)
ymin, ymax = np.min(las_ground_nonground.y), np.max(las_ground_nonground.y)

print(xmin, xmax, ymin, ymax)

629024.76 629148.17 5221512.49 5221598.18


## Check for different classifications so they can be excluded in the next step 

In [15]:
print(np.unique(las_ground_nonground.classification))

[0 1 2 7]


## Create DEM (IDW)

In [17]:
OUTPUT_TIF = WORK_DIR / "output_rast"

INPUT_LAS = OUTPUT_DIR / "las_ground_nonground.las"
DEM_FILE = OUTPUT_TIF / "raw_dem.tif"

print("Interpolating DEM...")
wbt.lidar_idw_interpolation(
i=str(INPUT_LAS),
output=str(DEM_FILE),
parameter="elevation",
returns="all",
resolution=0.2,
weight=1.0,
radius=2.5,
exclude_cls='0,1,7'
)

Interpolating DEM...
.\whitebox_tools.exe --run="LidarIdwInterpolation" --wd="d:\Lukas_B\Masterarbeit\preprocessing" --input='d:\Lukas_B\Masterarbeit\preprocessing\output_lidar\las_ground_nonground.las' --output='d:\Lukas_B\Masterarbeit\preprocessing\output_rast\raw_dem.tif' --parameter=elevation --returns=all --resolution=0.2 --weight=1.0 --radius=2.5 --exclude_cls='0,1,7' -v --compress_rasters=False

************************************
* Welcome to LidarIdwInterpolation *
* Powered by WhiteboxTools         *
* www.whiteboxgeo.com              *
************************************
Performing interpolation...
reading input LiDAR file...
Binning points: 0%
Binning points: 1%
Binning points: 2%
Binning points: 3%
Binning points: 4%
Binning points: 5%
Binning points: 6%
Binning points: 7%
Binning points: 8%
Binning points: 9%
Binning points: 10%
Binning points: 11%
Binning points: 12%
Binning points: 13%
Binning points: 14%
Binning points: 15%
Binning points: 16%
Binning points: 17%
Bin

0

## Create DSM (IDW)

In [18]:
DSM_FILE = OUTPUT_TIF / "raw_dsm.tif"

print("Interpolating DSM...")
wbt.lidar_idw_interpolation(
i=str(INPUT_LAS),
output=str(DSM_FILE),
parameter="elevation",
returns="first",
resolution=0.2,
weight=1.0,
radius=2.5
)

Interpolating DSM...
.\whitebox_tools.exe --run="LidarIdwInterpolation" --wd="d:\Lukas_B\Masterarbeit\preprocessing" --input='d:\Lukas_B\Masterarbeit\preprocessing\output_lidar\las_ground_nonground.las' --output='d:\Lukas_B\Masterarbeit\preprocessing\output_rast\raw_dsm.tif' --parameter=elevation --returns=first --resolution=0.2 --weight=1.0 --radius=2.5 -v --compress_rasters=False

************************************
* Welcome to LidarIdwInterpolation *
* Powered by WhiteboxTools         *
* www.whiteboxgeo.com              *
************************************
Performing interpolation...
reading input LiDAR file...
Binning points: 0%
Binning points: 1%
Binning points: 2%
Binning points: 3%
Binning points: 4%
Binning points: 5%
Binning points: 6%
Binning points: 7%
Binning points: 8%
Binning points: 9%
Binning points: 10%
Binning points: 11%
Binning points: 12%
Binning points: 13%
Binning points: 14%
Binning points: 15%
Binning points: 16%
Binning points: 17%
Binning points: 18%
Bin

0

## Create CHM

In [20]:
CHM_FILE = OUTPUT_TIF / "raw_chm.tif"

print("Creating CHM...")
wbt.subtract(
    input1=str(DSM_FILE),
    input2=str(DEM_FILE),
    output=str(CHM_FILE)
)

Creating CHM...
.\whitebox_tools.exe --run="Subtract" --wd="d:\Lukas_B\Masterarbeit\preprocessing" --input1='d:\Lukas_B\Masterarbeit\preprocessing\output_rast\raw_dsm.tif' --input2='d:\Lukas_B\Masterarbeit\preprocessing\output_rast\raw_dem.tif' --output='d:\Lukas_B\Masterarbeit\preprocessing\output_rast\raw_chm.tif' -v --compress_rasters=False

****************************
* Welcome to Subtract      *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35

0

# Define CRS for DTM,DSM and CHM

In [21]:
with rasterio.open(str(DEM_FILE), "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)

with rasterio.open(str(DSM_FILE), "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)

with rasterio.open(str(CHM_FILE), "r+") as dataset:
    dataset.crs = "EPSG:32632"
    print(dataset.crs)


EPSG:32632
EPSG:32632
EPSG:32632


# Normalize Pointcloud

In [22]:
INPUT_LAS = OUTPUT_DIR / "nonground_csf.las"
NORM_LAS = OUTPUT_DIR / "normalized_pc.las"

wbt.normalize_lidar(
    i=str(INPUT_LAS),
    output=str(NORM_LAS),
    dtm=str(DEM_FILE)
)

.\whitebox_tools.exe --run="NormalizeLidar" --wd="d:\Lukas_B\Masterarbeit\preprocessing" --input='d:\Lukas_B\Masterarbeit\preprocessing\output_lidar\nonground_csf.las' --output='d:\Lukas_B\Masterarbeit\preprocessing\output_lidar\normalized_pc.las' --dtm='d:\Lukas_B\Masterarbeit\preprocessing\output_rast\raw_dem.tif' -v --compress_rasters=False

*****************************
* Welcome to NormalizeLidar *
* Powered by WhiteboxTools  *
* www.whiteboxgeo.com       *
*****************************
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
Progress:

0

In [ ]:
las_normalized = laspy.read(str(NORM_LAS))
np.min(las_normalized.z)

np.float64(-2.1)

# Before continuing run treeiso.py in command line on non ground pointcloud to segment individual trees!!!

In [30]:
INPUT_LAS = OUTPUT_DIR / "nonground_csf_treeiso.laz"
las_trees = laspy.read(str(INPUT_LAS))

print(las_trees.header.vlrs)

[<laspy.vlrs.known.WktCoordinateSystemVlr object at 0x000001EEE891BCE0>, <VLR(user_id: 'liblas', record_id: '2112', data len: 597)>, <ExtraBytesVlr(extra bytes structs: 3)>]
